This notebook will be used for the capstone project

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import requests
import matplotlib.pyplot as plt
import matplotlib.cm as cm 
import matplotlib.colors as colors
import pgeocode
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup

Make a request to get the content of the website


In [2]:
webpage_response = requests.get("http://en.turkcewiki.org/wiki/List_of_postal_codes_of_Canada:_M")
webpage = webpage_response.content
#Create a BeautifulSoup object
soup =BeautifulSoup(webpage,"html.parser")

Create a empty list and find the table data within the webpage 

In [3]:
table_contents = []
#Find the table data
table = soup.find('table')

Iterate through the table data, here create a dictionary called cell having three keys: PostalCode, Borough and Neighborhood.

The postcode value will be extrated from row.p.text. The borough and Neighborhood will be extracted from row.span.txt
Cell will be conformed only where the borough and the neigborhoods are assigned. 

In the table, the neighborhood are inside parentheses and if the are more than one neighborhhood for one postal code they are separated by a /. To obtain the neighborhoods without the parentheses and separated by a comma, use the split and replace methods.

In [4]:
for row in table.findAll('td'):
    #Create a dictionary with 3 keys: PostalCode, Borough and Neighborhood
    cell = {}
    #Not including "Not assigned" values
    if row.span.text=='Not assigned':
        pass
    else:
        #Get the 3 PostalCode Letters
        cell['PostalCode'] = row.p.text[:3]
        #Get the Borough
        cell['Borough'] = (row.span.text).split('(')[0]
        #Get the neighborhoods, separate them with a ","
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

Created the dataframe and clean the data 

In [5]:
#Create the dataframe        
df=pd.DataFrame(table_contents)
#Rename the Borough values
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
print(df.head())
df.shape

  PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government


(103, 3)

Obtain the latitudes and longitudes for every PostalCode using the pgeocode module

In [6]:
geolocator = pgeocode.Nominatim("CA")
latitudes = []
longitudes = []
for code in df["PostalCode"]    :          
    location=geolocator.query_postal_code(code) 
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)
df["Latitude"] = latitudes
df["Longitude"] = longitudes
df.head()



,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889


Define your Foursquare credentials

In [7]:
CLIENT_ID = 'PWJUI1C335YAEP5EZ2U0PR1K4AV5Z51BFC1LIOG5OGFCUXGC' # your Foursquare ID
CLIENT_SECRET = 'PM2FU40TUSHEIZ20ZOVOGC3GY4M2BK3SAJZ5EKNOEUV05YUV' # your Foursquare Secret
ACCESS_TOKEN = 'FEXV1SH3YRZIGPV1C3QA2KB21XRF510AALHTRBAHMWFS0L5P' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: PWJUI1C335YAEP5EZ2U0PR1K4AV5Z51BFC1LIOG5OGFCUXGC
CLIENT_SECRET:PM2FU40TUSHEIZ20ZOVOGC3GY4M2BK3SAJZ5EKNOEUV05YUV


I decided to analyze all the Boroughs that the word "York" in them.

In [8]:
york_data = df[df['Borough'].astype(str).str.contains('York')].reset_index(drop=True)
york_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
3,M3B,North York,Don Mills North,43.7450,-79.3590
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094


Get the nearby venues for all the neighborhoods using the function from the New York Clustering lab. I will find venues within a radius of 1000m.

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(\
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
york_venues = getNearbyVenues(york_data['Neighborhood'],
                              york_data['Latitude'],
                              york_data['Longitude'])


Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills North
Parkview Hill, Woodbine Gardens
Glencairn
Don Mills South
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Bayview Village
Downsview East
York Mills, Silver Hills
Downsview West
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview Central
Bedford Park, Lawrence Manor East
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Humberlea, Emery
Willowdale South
Downsview Northwest
Runnymede, The Junction North
Weston
York Mills West
Willowdale West


In [11]:
print(york_venues.shape)
york_venues.head()

(729, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7545,-79.33,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.7545,-79.33,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.7545,-79.33,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.7545,-79.33,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.7545,-79.33,Food Basics,43.760549,-79.326045,Supermarket


The venues per neigborhood are:

In [12]:
york_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",30,30,30,30,30,30
Bayview Village,8,8,8,8,8,8
"Bedford Park, Lawrence Manor East",30,30,30,30,30,30
Caledonia-Fairbanks,24,24,24,24,24,24
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",14,14,14,14,14,14
Don Mills North,12,12,12,12,12,12
Don Mills South,30,30,30,30,30,30
Downsview Central,8,8,8,8,8,8
Downsview East,24,24,24,24,24,24


We can see that Willowdale South has the most venues. Now, we can see how many unique venue categories are

In [13]:
print("The are "+str(len(york_venues["Venue Category"].unique())) +" venue categories.")

The are 156 venue categories.


Next, perform one hot encoding to the York venues.

In [14]:
york_one_hot = pd.get_dummies(york_venues[["Venue Category"]])
york_one_hot["Neighborhood"] = york_venues["Neighborhood"]
# move neighborhood column to the first column
fixed_columns = [york_one_hot.columns[-1]] + list(york_one_hot.columns[:-1])
york_one_hot = york_one_hot[fixed_columns]
york_one_hot.head()

,Neighborhood,Venue Category_ATM,Venue Category_Afghan Restaurant,Venue Category_Airport,Venue Category_American Restaurant,Venue Category_Argentinian Restaurant,Venue Category_Art Gallery,Venue Category_Arts & Crafts Store,Venue Category_Asian Restaurant,Venue Category_Athletics & Sports,Venue Category_Auto Dealership,Venue Category_BBQ Joint,Venue Category_Baby Store,Venue Category_Bagel Shop,Venue Category_Bakery,Venue Category_Bank,Venue Category_Bar,Venue Category_Baseball Field,Venue Category_Basketball Court,Venue Category_Beer Bar,Venue Category_Beer Store,Venue Category_Bike Shop,Venue Category_Bookstore,Venue Category_Breakfast Spot,Venue Category_Brewery,Venue Category_Bridal Shop,Venue Category_Bridge,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Bus Line,Venue Category_Bus Station,Venue Category_Bus Stop,Venue Category_Business Service,Venue Category_Butcher,Venue Category_Café,Venue Category_Cantonese Restaurant,Venue Category_Caribbean Restaurant,Venue Category_Chinese Restaurant,Venue Category_Chocolate Shop,Venue Category_Climbing Gym,Venue Category_Clothing Store,Venue Category_Coffee Shop,Venue Category_Comfort Food Restaurant,Venue Category_Construction & Landscaping,Venue Category_Convenience Store,Venue Category_Cosmetics Shop,Venue Category_Curling Ice,Venue Category_Deli / Bodega,Venue Category_Department Store,Venue Category_Dessert Shop,Venue Category_Dim Sum Restaurant,Venue Category_Diner,Venue Category_Discount Store,Venue Category_Drugstore,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store,Venue Category_Ethiopian Restaurant,Venue Category_Falafel Restaurant,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant,Venue Category_Field,Venue Category_Fish & Chips Shop,Venue Category_Flea Market,Venue Category_Food & Drink Shop,Venue Category_Food Court,Venue Category_French Restaurant,Venue Category_Fried Chicken Joint,Venue Category_Frozen Yogurt Shop,Venue Category_Furniture / Home Store,Venue Category_Gas Station,Venue Category_Gastropub,Venue Category_General Entertainment,Venue Category_Go Kart Track,Venue Category_Golf Course,Venue Category_Gourmet Shop,Venue Category_Greek Restaurant,Venue Category_Grocery Store,Venue Category_Gym,Venue Category_Gym / Fitness Center,Venue Category_History Museum,Venue Category_Hockey Arena,Venue Category_Hookah Bar,Venue Category_Hostel,Venue Category_Hotel,Venue Category_Ice Cream Shop,Venue Category_Indian Restaurant,Venue Category_Intersection,Venue Category_Irish Pub,Venue Category_Italian Restaurant,Venue Category_Japanese Restaurant,Venue Category_Jewelry Store,Venue Category_Juice Bar,Venue Category_Kitchen Supply Store,Venue Category_Korean Restaurant,Venue Category_Latin American Restaurant,Venue Category_Laundromat,Venue Category_Leather Goods Store,Venue Category_Liquor Store,Venue Category_Lounge,Venue Category_Massage Studio,Venue Category_Mediterranean Restaurant,Venue Category_Men's Store,Venue Category_Metro Station,Venue Category_Mexican Restaurant,Venue Category_Middle Eastern Restaurant,Venue Category_Mobile Phone Shop,Venue Category_Movie Theater,Venue Category_Moving Target,Venue Category_New American Restaurant,Venue Category_Nightclub,Venue Category_Optical Shop,Venue Category_Paintball Field,Venue Category_Park,Venue Category_Pet Store,Venue Category_Pharmacy,Venue Category_Photography Lab,Venue Category_Pizza Place,Venue Category_Playground,Venue Category_Plaza,Venue Category_Pool,Venue Category_Portuguese Restaurant,Venue Category_Pub,Venue Category_Racetrack,Venue Category_Ramen Restaurant,Venue Category_Rental Service,Venue Category_Residential Building (Apartment / Condo),Venue Category_Restaurant,Venue Category_Rock Climbing Spot,Venue Category_Salon / Barbershop,Venue Category_Sandwich Place,Venue Category_Seafood Restaurant,Venue Category_Shopping Mall,Venue Category_Skating Rink,Venue Category_Smoothie Shop,Venue Category_Soccer Field,Venue Category_Soccer Stadium,V

Group by the neighborhood, obtain the mean of the frequency of every venue and reset the index

In [15]:
york_group = york_one_hot.groupby("Neighborhood").mean().reset_index()
york_group.head()

,Neighborhood,Venue Category_ATM,Venue Category_Afghan Restaurant,Venue Category_Airport,Venue Category_American Restaurant,Venue Category_Argentinian Restaurant,Venue Category_Art Gallery,Venue Category_Arts & Crafts Store,Venue Category_Asian Restaurant,Venue Category_Athletics & Sports,Venue Category_Auto Dealership,Venue Category_BBQ Joint,Venue Category_Baby Store,Venue Category_Bagel Shop,Venue Category_Bakery,Venue Category_Bank,Venue Category_Bar,Venue Category_Baseball Field,Venue Category_Basketball Court,Venue Category_Beer Bar,Venue Category_Beer Store,Venue Category_Bike Shop,Venue Category_Bookstore,Venue Category_Breakfast Spot,Venue Category_Brewery,Venue Category_Bridal Shop,Venue Category_Bridge,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Bus Line,Venue Category_Bus Station,Venue Category_Bus Stop,Venue Category_Business Service,Venue Category_Butcher,Venue Category_Café,Venue Category_Cantonese Restaurant,Venue Category_Caribbean Restaurant,Venue Category_Chinese Restaurant,Venue Category_Chocolate Shop,Venue Category_Climbing Gym,Venue Category_Clothing Store,Venue Category_Coffee Shop,Venue Category_Comfort Food Restaurant,Venue Category_Construction & Landscaping,Venue Category_Convenience Store,Venue Category_Cosmetics Shop,Venue Category_Curling Ice,Venue Category_Deli / Bodega,Venue Category_Department Store,Venue Category_Dessert Shop,Venue Category_Dim Sum Restaurant,Venue Category_Diner,Venue Category_Discount Store,Venue Category_Drugstore,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store,Venue Category_Ethiopian Restaurant,Venue Category_Falafel Restaurant,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant,Venue Category_Field,Venue Category_Fish & Chips Shop,Venue Category_Flea Market,Venue Category_Food & Drink Shop,Venue Category_Food Court,Venue Category_French Restaurant,Venue Category_Fried Chicken Joint,Venue Category_Frozen Yogurt Shop,Venue Category_Furniture / Home Store,Venue Category_Gas Station,Venue Category_Gastropub,Venue Category_General Entertainment,Venue Category_Go Kart Track,Venue Category_Golf Course,Venue Category_Gourmet Shop,Venue Category_Greek Restaurant,Venue Category_Grocery Store,Venue Category_Gym,Venue Category_Gym / Fitness Center,Venue Category_History Museum,Venue Category_Hockey Arena,Venue Category_Hookah Bar,Venue Category_Hostel,Venue Category_Hotel,Venue Category_Ice Cream Shop,Venue Category_Indian Restaurant,Venue Category_Intersection,Venue Category_Irish Pub,Venue Category_Italian Restaurant,Venue Category_Japanese Restaurant,Venue Category_Jewelry Store,Venue Category_Juice Bar,Venue Category_Kitchen Supply Store,Venue Category_Korean Restaurant,Venue Category_Latin American Restaurant,Venue Category_Laundromat,Venue Category_Leather Goods Store,Venue Category_Liquor Store,Venue Category_Lounge,Venue Category_Massage Studio,Venue Category_Mediterranean Restaurant,Venue Category_Men's Store,Venue Category_Metro Station,Venue Category_Mexican Restaurant,Venue Category_Middle Eastern Restaurant,Venue Category_Mobile Phone Shop,Venue Category_Movie Theater,Venue Category_Moving Target,Venue Category_New American Restaurant,Venue Category_Nightclub,Venue Category_Optical Shop,Venue Category_Paintball Field,Venue Category_Park,Venue Category_Pet Store,Venue Category_Pharmacy,Venue Category_Photography Lab,Venue Category_Pizza Place,Venue Category_Playground,Venue Category_Plaza,Venue Category_Pool,Venue Category_Portuguese Restaurant,Venue Category_Pub,Venue Category_Racetrack,Venue Category_Ramen Restaurant,Venue Category_Rental Service,Venue Category_Residential Building (Apartment / Condo),Venue Category_Restaurant,Venue Category_Rock Climbing Spot,Venue Category_Salon / Barbershop,Venue Category_Sandwich Place,Venue Category_Seafood Restaurant,Venue Category_Shopping Mall,Venue Category_Skating Rink,Venue Category_Smoothie Shop,Venue Category_Soccer Field,Venue Category_Soccer Stadium,V

Using the function defined on the lab, obtain the top 20 venues for each neigborhood.

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = york_group['Neighborhood']

for ind in np.arange(york_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Venue Category_Coffee Shop,Venue Category_Gas Station,Venue Category_Park,Venue Category_Pizza Place,Venue Category_Bank,Venue Category_Restaurant,Venue Category_Shopping Mall,Venue Category_Sandwich Place,Venue Category_Diner,Venue Category_Bridal Shop,Venue Category_Mediterranean Restaurant,Venue Category_Bus Line,Venue Category_Deli / Bodega,Venue Category_Men's Store,Venue Category_Middle Eastern Restaurant,Venue Category_Mobile Phone Shop,Venue Category_Intersection,Venue Category_Pharmacy,Venue Category_American Restaurant,Venue Category_Frozen Yogurt Shop
1,Bayview Village,Venue Category_Park,Venue Category_Golf Course,Venue Category_Chinese Restaurant,Venue Category_Café,Venue Category_Trail,Venue Category_Bank,Venue Category_Japanese Restaurant,Venue Category_Yoga Studio,Venue Category_Dessert Shop,Venue Category_Eastern European Restaurant,Venue Category_Drugstore,Venue Category_Discount Store,Venue Category_Diner,Venue Category_Dim Sum Restaurant,Venue Category_Deli / Bodega,Venue Category_Department Store,Venue Category_Ethiopian Restaurant,Venue Category_Curling Ice,Venue Category_Cosmetics Shop,Venue Category_Convenience Store
2,"Bedford Park, Lawrence Manor East",Venue Category_Coffee Shop,Venue Category_Italian Restaurant,Venue Category_Sandwich Place,Venue Category_Pharmacy,Venue Category_Sports Club,Venue Category_Café,Venue Category_Butcher,Venue Category_Liquor Store,Venue Category_Pub,Venue Category_Juice Bar,Venue Category_Bridal Shop,Venue Category_Restaurant,Venue Category_Fast Food Restaurant,Venue Category_Indian Restaurant,Venue Category_Bank,Venue Category_Bakery,Venue Category_Comfort Food Restaurant,Venue Category_Bagel Shop,Venue Category_Thai Restaurant,Venue Category_Sushi Restaurant
3,Caledonia-Fairbanks,Venue Category_Park,Venue Category_Mexican Restaurant,Venue Category_Pizza Place,Venue Category_Coffee Shop,Venue Category_Pharmacy,Venue Category_Bakery,Venue Category_Cosmetics Shop,Venue Category_Portuguese Restaurant,Venue Category_Rental Service,Venue Category_Bus Line,Venue Category_Discount Store,Venue Category_Beer Store,Venue Category_Sporting Goods Shop,Venue Category_Bank,Venue Category_Japanese Restaurant,Venue Category_Wings Joint,Venue Category_BBQ Joint,Venue Category_Hostel,Venue Category_Women's Store,Venue Category_Grocery Store
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Venue Category_Coffee Shop,Venue Category_Furniture / Home Store,Venue Category_Intersection,Venue Category_Grocery Store,Venue Category_Pizza Place,Venue Category_Discount Store,Venue Category_Park,Venue Category_Fast Food Restaurant,Venue Category_Gas Station,Venue Category_Department Store,Venue Category_Diner,Venue Category_Dim Sum Restaurant,Venue Category_Dessert Shop,Venue Category_Cosmetics Shop,Venue Category_Deli / Bodega,Venue Category_Curling Ice,Venue Category_Eastern European Restaurant,Venue Category_Convenience Store,Venue Category_Construction & Landscaping,Venue Category_Comfort Food Restaurant


K-means clustering: Use k = 15

In [18]:
kclusters = 15

york_cluster = york_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_cluster)


Finally, create a dataframe with the top 15 venues with the cluster value and add it to the initial data

In [19]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

york_final = york_data
york_final = york_final.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
york_final 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,6,Venue Category_Park,Venue Category_Bus Stop,Venue Category_ATM,Venue Category_Tennis Court,Venue Category_Food & Drink Shop,Venue Category_Fast Food Restaurant,Venue Category_Coffee Shop,Venue Category_Chinese Restaurant,Venue Category_Pharmacy,Venue Category_Caribbean Restaurant,Venue Category_Café,Venue Category_Shopping Mall,Venue Category_Supermarket,Venue Category_Pizza Place,Venue Category_Train Station,Venue Category_Baby Store,Venue Category_Dessert Shop,Venue Category_Electronics Store,Venue Category_Eastern European Restaurant,Venue Category_Drugstore
1,M4A,North York,Victoria Village,43.7276,-79.3148,11,Venue Category_Coffee Shop,Venue Category_Park,Venue Category_Pizza Place,Venue Category_Hockey Arena,Venue Category_Sporting Goods Shop,Venue Category_Portuguese Restaurant,Venue Category_Thrift / Vintage Store,Venue Category_Intersection,Venue Category_Middle Eastern Restaurant,Venue Category_Deli / Bodega,Venue Category_Discount Store,Venue Category_Diner,Venue Category_Dim Sum Restaurant,Venue Category_Dessert Shop,Venue Category_Department Store,Venue Category_Cosmetics Shop,Venue Category_Curling Ice,Venue Category_Eastern European Restaurant,Venue Category_Convenience Store,Venue Category_Construction & Landscaping
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,3,Venue Category_Clothing Store,Venue Category_Cosmetics Shop,Venue Category_Jewelry Store,Venue Category_Pharmacy,Venue Category_Movie Theater,Venue Category_Dessert Shop,Venue Category_Men's Store,Venue Category_Mediterranean Restaurant,Venue Category_Restaurant,Venue Category_Leather Goods Store,Venue Category_Shopping Mall,Venue Category_Bookstore,Venue Category_Coffee Shop,Venue Category_Kitchen Supply Store,Venue Category_Chocolate Shop,Venue Category_Fried Chicken Joint,Venue Category_Food Court,Venue Category_Greek Restaurant,Venue Category_American Restaurant,Venue Category_Furniture / Home Store
3,M3B,North York,Don Mills North,43.7450,-79.3590,6,Venue Category_Pizza Place,Venue Category_Diner,Venue Category_Coffee Shop,Venue Category_Soccer Field,Venue Category_Park,Venue Category_Sandwich Place,Venue Category_Gas Station,Venue Category_Juice Bar,Venue Category_Athletics & Sports,Venue Category_Burger Joint,Venue Category_Restaurant,Venue Category_Bank,Venue Category_Discount Store,Venue Category_Dessert Shop,Venue Category_Dim Sum Restaurant,Venue Category_Eastern European Restaurant,Venue Category_Department Store,Venue Category_Deli / Bodega,Venue Category_Curling Ice,Venue Category_Cosmetics Shop
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094,7,Venue Category_Pizza Place,Venue Category_Coffee Shop,Venue Category_Brewery,Venue Category_Bank,Venue Category_Gastropub,Venue Category_Intersection,Venue Category_Flea Market,Venue Category_Fast Food Restaurant,Venue Category_Construction & Landscaping,Venue Category_Pharmacy,Venue Category_Café,Venue Category_Rock Climbing Spot,Venue Category_Breakfast Spot,Venue Category_Soccer Stadium,Venue Category_Gym / Fitness Center,Venue Category_Bakery,Venue Category_Athletics & Sports,Venue Category_Argentinian Restaurant,Venue Category_Curling Ice,Venue Category_Deli / Bodega
5,M6B,North York,Glencairn,43.7081,-79.4479,0,Venue Category_Grocery Store,Venue Category_Fast Food Restaurant,Venue Category_Coffee Shop,Venue Category_Gas Station,Venue Category_Pizza Place,Venue Category_Bank,Venue Category_Bakery,Ve

Finally, visualize the clusters

In [20]:
# create map
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
lat = location.latitude
long = location.longitude
map_clusters = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_final['Latitude'], york_final['Longitude'], york_final['Neighborhood'], york_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters